In [14]:
## Init stuff
import mujoco
import mujoco.viewer as viewer
import numpy as np
import time
import matplotlib.pyplot as plt

## Joint lengths
kp = 0
kd = 0
l1 = 0.2
l2 = 0.5
l3 = 0.5

# Raise the robot up a bit so it doesn't clip into the floor
base_pos_x, base_pos_y, base_pos_z = 0, 0, 0.1
ball_pos_x, ball_pos_y, ball_pos_z = 0.2, -0.4, 0.2
maze_pos_x, maze_pos_y, maze_pos_z = 0, 0, 0

### TODO: look up tuning? dampratio="1" might already tune kv automatically
### but kp still needs to be tuned properly
### https://en.wikipedia.org/wiki/Ziegler%E2%80%93Nichols_method
kp = 200

robot_4dof = f'''
<mujoco model="3dof_arm">
    <compiler angle="radian" coordinate="local" meshdir="assets" autolimits="true"/>
    <option gravity="0 0 -9.81"/>

    <asset>
        <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
        rgb2=".2 .3 .4" width="300" height="300"/>
        <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
        <mesh name="maze" file="MazeIII.STL" scale="0.005 0.005 0.005"/>
        <mesh name="ee" file="ee1.STL" scale="0.05 0.05 0.05"/>
    </asset>
    
    <worldbody>
        <geom size="2 2 0.01" type="plane" material="grid"/>
        <light pos="0 0 1"/>
        
        <!-- Ball -->
        <body name="ball" pos="{ball_pos_x} {ball_pos_y} {ball_pos_z}">
            <geom type="sphere" size="0.03" rgba="1 1 0 1"/>
            <joint type="free"/>
        </body>

        <!-- Maze -->
        <body name="maze" pos="{maze_pos_x} {maze_pos_y} {maze_pos_z}">
            <geom type="mesh" mesh="maze" name="maze_geom" rgba="0.7 0.7 0.7 1" euler="1.5708 0 0"/>
        </body>
            
        <!-- Base -->
        <body name="base" pos="{base_pos_x} {base_pos_y} {base_pos_z}">
            
            <!-- Joint 1 -->
            <body name="joint1" pos="{base_pos_x} {base_pos_y} {base_pos_z}">
                <joint name="joint1_hinge" type="hinge" axis="0 0 1"/>
                <geom type="sphere" size="0.05" rgba="1 0 0 1"/>
                
                <!-- Link 1 -->
                <geom type="cylinder" fromto="0 0 0 0 0 {l1}" size="0.02" rgba="0 1 0 1"/>
                
                <!-- Joint 2 -->
                <body name="joint2" pos="0 0 {l1}">
                    <joint name="joint2_hinge" type="hinge" axis="0 1 0"/>
                    <geom type="sphere" size="0.05" rgba="1 0 0 1"/>
                    
                    <!-- Link 2 -->
                    <geom type="cylinder" fromto="0 0 0 0 0 {l2}" size="0.02" rgba="0 1 0 1"/>
                    
                    <!-- Joint 3 -->
                    <body name="joint3" pos="0 0 {l2}">
                        <joint name="joint3_hinge" type="hinge" axis="0 1 0"/>
                        <geom type="sphere" size="0.05" rgba="1 0 0 1"/>
                        
                        <!-- Link 3 -->
                        <geom type="cylinder" fromto="0 0 0 0 0 {l3}" size="0.02" rgba="0 1 0 1"/>
                        
                        <!-- End-effector -->
                        <body name="ee" pos="0 0 {l3}">
                            <joint name="joint4_hinge" type="hinge" axis="0 1 0"/>
                            <geom type="sphere" size="0.02" rgba="1 1 1 1"/>
                            <geom type="mesh" mesh="ee" rgba="1 1 1 1"/>
                            
                        </body>
                    </body>
                </body>
            </body>
        </body>
    </worldbody>

    <actuator>
        <position joint="joint1_hinge" kp="{kp}" dampratio="1"/>
        <position joint="joint2_hinge" kp="{kp}" dampratio="1"/>
        <position joint="joint3_hinge" kp="{kp}" dampratio="1"/>
        <position joint="joint4_hinge" kp="1" dampratio="1"/>
    </actuator>
</mujoco>
'''

#     <body name="ee" pos="0 0 0" gravecomp="1">
#         <geom mesh="ee" size="0.03" rgba="1 1 1 1"/>

# Note: different end effectors need different dampings
#TODO: implement end effector

In [ ]:
## Simulation with joint positions controllable using GUI
## Use this to manually control the joints and do sanity checks

# Load model to reset robot
model = mujoco.MjModel.from_xml_string(robot_4dof)
data = mujoco.MjData(model)

with viewer.launch(model, data) as v:
    while v.is_running():
        mujoco.mj_step(model, data)
        v.render()

In [16]:
## IK

def shortest_angle_update(theta_current, theta_dest, scale=1):
    # Finds closest angle configuration equivalent of goal configuration
    # to current configuration
    #  
    # theta_current: current angle measure
    # theta_dest: desired angle measure
    # scale: scale factor of the step
    #        Change scale to overshoot or undershoot (in case of error)
    #        Probably useless but maybe fun to toy with?
    # returns the closest angle equivalent of theta_dest to theta_current
    diff = theta_dest - theta_current
    diff = (diff + np.pi) % (2*np.pi) - np.pi
    return theta_current + scale * diff

def dist(point1, point2):
    # Finds distance between two points 
    # returns distance between the two points
    return np.linalg.norm(np.array(point2) - np.array(point1))

def forward_kinematics(theta1, theta2, theta3, l1, l2, l3):
    # Calculates the ee position given joint angles and arm lengths
    # Returns np.array([x, y, z]) of coordinates
    s1 = np.sin(theta1)
    s2 = np.sin(theta2)
    s3 = np.sin(theta3)
    c1 = np.cos(theta1)
    c2 = np.cos(theta2)
    c3 = np.cos(theta3)
    x = -(l3*(c1*c2*c3 - c1*s2*s3) + c1*c2*l2)
    y = -(l3*(-s1*c2*c3 + s1*s2*s3) - s1*c2*l2)
    z = l3*(s2*c3 + c2*s3) + s2*l2 + l1

    return np.array([x, y, z])


# Load model
model = mujoco.MjModel.from_xml_string(robot_4dof)
data = mujoco.MjData(model)

# Init parameters
ee_id = model.body('ee').id
base_id = model.body('base').id
ball_id = model.body('ball').id
base_xpos = data.body(base_id).xpos
tol = 1e-4
update_freq = 1/60 # framerate/simulation rate. lower to speed up

# Currently unused
dest = np.array([0.7, -0.1, 0.1])

with mujoco.viewer.launch_passive(model, data) as v:
    while v.is_running():
        # Direction algorithm should go around here
        ball_pos = data.body(ball_id).xpos
        ee_pos = data.body(ee_id).xpos
        # error = dist(ball_pos, ee_pos)
        error = 100 # make the ee always follow the ball
        if error > tol:
            goal = ball_pos
        else:
            goal = dest

        # Account for base position
        x_offset, y_offset, z_offset = np.subtract(goal, base_xpos)

        # IK from previous milestone
        theta_curr = data.qpos.copy()
        theta1 = np.arctan2(-y_offset, -x_offset)
        r = np.sqrt(x_offset**2 + y_offset**2)
        D = (r**2 + z_offset**2 - l2**2 - l3**2) / (2 * l2 * l3)
        theta3 = -np.arccos((x_offset**2 + y_offset**2 + z_offset**2 -l2**2-l3**2)/(2*l2*l3))
        theta2 = np.arcsin(z_offset/np.sqrt(x_offset**2 + y_offset**2 + z_offset**2)) + np.arctan2(l3*np.sin(theta3), l2+l3*np.cos(theta3))
        theta4 = np.pi - (theta2 + theta3)

        # Workaround for snapping between pi and -pi
        theta1_adj = shortest_angle_update(theta_curr[0], theta1)
        theta2_adj = shortest_angle_update(theta_curr[1], theta2)
        theta3_adj = shortest_angle_update(theta_curr[2], theta3)
        theta4_adj = np.pi-(theta2_adj + theta3_adj)

        # FK for sanity check and debugging
        ee_fk = forward_kinematics(theta1_adj, theta2_adj, theta3_adj, l1, l2, l3)
        # print(data.body(ee_id).xpos, goal, error)
        ##BUG: z position off. maybe something to do with gravcomp/pd values?

        # Send commands to the robot
        q = np.array([theta1_adj, theta2_adj, theta3_adj, theta4_adj])
        # q = np.array([0, 0, 0, 0])
        data.ctrl = q
        mujoco.mj_step(model, data)
        v.sync()
        time.sleep(update_freq)

C:\Users\Nelson\AppData\Local\Temp\ipykernel_5424\1475677695.py:73: RuntimeWarning: invalid value encountered in arccos
  theta3 = -np.arccos((x_offset**2 + y_offset**2 + z_offset**2 -l2**2-l3**2)/(2*l2*l3))


Load model

define trajectory calculation
inputs current ball pos, goal pos
returns directions

run simulation
while simulation is running:
    new traj = true
    if new traj:
        calculate trajectory -> get directions
        position arm at ball pos (lift arm -> move -> lower arm)
        * make sure the arm doesn't swing too much
        * take steps of max distance D until it reaches tolerable distance from point
        * maybe add force sensing to tell robot to stop lowering?
        new traj = false

    for each step in directions
        calculate next location
        (if position control):
            move to location
        (if velocity/force control):
            calculate distance between current location and next location 
            step towards location (size of step depends on distance)

    check ball pos and ee pos
    if dist(ball pos, ee pos) > tolerance:
        redo trajectory calculation
        new traj = true

TODO: detect if goal outside workspace